In [1]:
from VariationalAutoDecoder import VariationalAutoDecoder as VAD
from VAD_Trainer import VAD_Trainer
import utils
from evaluate import evaluate_model
import torch
import torch.optim as optim
import torch.nn as nn
import csv
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Create DataLoaders

In [2]:
train_ds, train_dl, test_ds, test_dl = utils.create_dataloaders(data_path="dataset", batch_size=64)

## Train Auto Decoder

In [5]:
num_test_samples = len(test_dl.dataset)
vad = VAD()
trainer = VAD_Trainer(var_decoder=vad, dataloader=train_dl, latent_dim=256, device=device, lr=1e-3)
latents = torch.nn.Parameter(torch.randn(num_test_samples, trainer.latent_dim).to(device))
optimizer = optim.Adam([latents], lr=1e-3)

In [6]:
start_time = time.time()  # Record the start time
train_loss = trainer.train(num_epochs=200)  # Train the model
end_time = time.time()  # Record the end time

elapsed_time = end_time - start_time  # Calculate elapsed time
print(f"Trainer has finished training in {elapsed_time:.2f} seconds.")

# start_time = time.time()  # Record the start time
# test_loss = evaluate_model(model=vad, test_dl=test_dl, opt=optimizer, latents=latents, epochs=100, device=device) 
# end_time = time.time()  # Record the end time

# elapsed_time = end_time - start_time  # Calculate elapsed time
# print(f"VAD has finished test evaluation in {elapsed_time:.2f} seconds.")

Epoch [1/200], Loss: 389.1837
Epoch [2/200], Loss: 122.2750
Epoch [3/200], Loss: 64.8375
Epoch [4/200], Loss: 42.0015
Epoch [5/200], Loss: 30.1352
Epoch [6/200], Loss: 23.1528
Epoch [7/200], Loss: 18.6156
Epoch [8/200], Loss: 15.5062
Epoch [9/200], Loss: 13.3488
Epoch [10/200], Loss: 11.8795
Epoch [11/200], Loss: 10.9637
Epoch [12/200], Loss: 10.5592
Epoch [13/200], Loss: 10.7367
Epoch [14/200], Loss: 11.6681
Epoch [15/200], Loss: 13.4652
Epoch [16/200], Loss: 15.9846
Epoch [17/200], Loss: 20.0045
Epoch [18/200], Loss: 25.9952
Epoch [19/200], Loss: 26.4398
Epoch [20/200], Loss: 20.9384
Epoch [21/200], Loss: 12.6140
Epoch [22/200], Loss: 5.4882
Epoch [23/200], Loss: 2.6364
Epoch [24/200], Loss: 1.7940
Epoch [25/200], Loss: 1.4999
Epoch [26/200], Loss: 1.3474
Epoch [27/200], Loss: 1.2445
Epoch [28/200], Loss: 1.1658
Epoch [29/200], Loss: 1.1007
Epoch [30/200], Loss: 1.0441
Epoch [31/200], Loss: 0.9931
Epoch [32/200], Loss: 0.9508
Epoch [33/200], Loss: 0.9130
Epoch [34/200], Loss: 0.8795


In [10]:
architectures = [
    # 1. Simple Architecture (Latent space: 64)
    nn.Sequential(
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 784)
    ),
    
    # 2. Deeper Architecture (Latent space: 32)
    nn.Sequential(
        nn.Linear(32, 128),
        nn.LeakyReLU(),
        nn.Linear(128, 256),
        nn.LeakyReLU(),
        nn.Linear(256, 512),
        nn.LeakyReLU(),
        nn.Linear(512, 1024),
        nn.LeakyReLU(),
        nn.Linear(1024, 784)
    ),
    
    # 3. Wider Architecture (Latent space: 128)
    nn.Sequential(
        nn.Linear(128, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024, 784)
    ),
    
    # 4. Progressive Architecture (Latent space: 16)
    nn.Sequential(
        nn.Linear(16, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 784)
    ),
    
    # 5. Bottlenecked Architecture (Latent space: 10)
    nn.Sequential(
        nn.Linear(10, 64),
        nn.ReLU(),
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 784)
    )
]

latent_dims = [dim for dim in [64, 32, 128, 16, 10] for _ in range(5)]
auto_decoders = [AutoDecoder.AutoDecoder(arch) for arch in architectures for _ in range(5)]
learning_rates = [lr for lr in [0.001, 0.0005, 0.0001, 0.002, 0.005] for _ in range(5)]
trainers = [AD_Trainer.AD_Trainer(decoder=auto_decoders[i], dataloader=train_dl, latent_dim=latent_dims[i], device=device, lr=learning_rates[i]) for i in range(len(latent_dims))]

In [11]:
# Initialize the results list to hold all the data
num_test_samples = len(test_dl.dataset)

# Create latent parameters and optimizers for each trainer
latents_list = [torch.nn.Parameter(torch.randn(num_test_samples, trainers[5*i].latent_dim).to(device)) for i in range(5) for _ in range(5)]
optimizers = [optim.Adam([latents], lr=1e-3) for latents in latents_list]

# Save results to a CSV file
csv_file_path = 'results.csv'

# Write header to the CSV file first
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['Index'] + [f'Epoch {i+1} Loss' for i in range(200)] + ['Final Test Loss']
    writer.writerow(header)

# Main training and evaluation loop
for index, trainer in enumerate(trainers):
    start_time = time.time()  # Record the start time
    train_loss = trainer.train(num_epochs=200)  # Train the model
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"Trainer {index} has finished training in {elapsed_time:.2f} seconds.")

    start_time = time.time()  # Record the start time
    test_loss = evaluate_model(model=auto_decoders[index], test_dl=test_dl, opt=optimizers[index], latents=latents_list[index], epochs=100, device=device) 
    end_time = time.time()  # Record the end time
    
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"AD {index} has finished test evaluation in {elapsed_time:.2f} seconds.")

    # Prepare the row to be saved
    row = [index] + train_loss + [test_loss]

    # Append results to the CSV file after each iteration
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

print(f"Results saved to {csv_file_path}.")

Epoch [1/200], Loss: 0.5146
Epoch [2/200], Loss: 0.4237
Epoch [3/200], Loss: 0.3489
Epoch [4/200], Loss: 0.3360
Epoch [5/200], Loss: 0.3307
Epoch [6/200], Loss: 0.3288
Epoch [7/200], Loss: 0.3265
Epoch [8/200], Loss: 0.3244
Epoch [9/200], Loss: 0.3226
Epoch [10/200], Loss: 0.3208
Epoch [11/200], Loss: 0.3191
Epoch [12/200], Loss: 0.3174
Epoch [13/200], Loss: 0.3158
Epoch [14/200], Loss: 0.3143
Epoch [15/200], Loss: 0.3128
Epoch [16/200], Loss: 0.3113
Epoch [17/200], Loss: 0.3099
Epoch [18/200], Loss: 0.3086
Epoch [19/200], Loss: 0.3073
Epoch [20/200], Loss: 0.3060
Epoch [21/200], Loss: 0.3049
Epoch [22/200], Loss: 0.3038
Epoch [23/200], Loss: 0.3028
Epoch [24/200], Loss: 0.3019
Epoch [25/200], Loss: 0.3011
Epoch [26/200], Loss: 0.3005
Epoch [27/200], Loss: 0.2999
Epoch [28/200], Loss: 0.2995
Epoch [29/200], Loss: 0.2991
Epoch [30/200], Loss: 0.2989
Epoch [31/200], Loss: 0.2987
Epoch [32/200], Loss: 0.2986
Epoch [33/200], Loss: 0.2985
Epoch [34/200], Loss: 0.2984
Epoch [35/200], Loss: 0

## Evaluate Auto Decoder

In [ ]:
num_test_samples = len(test_dl.dataset)
latents = torch.nn.Parameter(torch.randn(num_test_samples, trainer.latent_dim).to(device))
opt = optim.Adam([latents], lr=1e-3)

In [ ]:
loss = evaluate_model(model=auto_decoder, test_dl=test_dl, opt=opt, latents=latents, epochs=100, device=device)
print(loss)